In [2]:
import cv2
import matplotlib.pyplot as plt
from glob import glob
import numpy as np
from math import ceil
from skimage import feature

In [6]:
class CoringDetector:

    def __init__(self, images: np.array) -> None:
        self.input_images = images
        self.all_transforms = {"input_images": self.input_images}

    def apply_transforms(
        self,
        canny_sigma: float = 2.7,
        N1: int = 19,
        N2: int = 9,
        N3: int = 5,
    ):
        images = self.input_images.copy()

        # Calcular bordas com canny
        images = [feature.canny(img, sigma=canny_sigma) * 255.0 for img in self.input_images]
        self.all_transforms["canny"] = images

        # Fechar bordas encontradas com fechamento (para criar blobs)
        kernel1 = np.ones((N1, N1), np.uint8)
        images = [cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel1) for img in images]
        self.all_transforms["closing"] = images

        # Aplicando abertura com kernel vertical para remover ruidos horizontais
        kernel2 = np.ones((N2, 1), np.uint8)
        images = [cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel2) for img in images]
        self.all_transforms["open"] = images

        # Aplicando dilatação para juntar ruidos proximos em grandes regiões, pode facilitar
        # na filtragem por área, mas talvez não seja necessario.
        kernel3 = np.ones((N3, N3), np.uint8)
        images = [cv2.morphologyEx(img, cv2.MORPH_DILATE, kernel3) for img in images]
        self.all_transforms["dialate"]

        return images

    def find_contours(self, images: list):

        converted_images = []
        for image in np.array(images):
            # Ensure that the image is in uint8 format (CV_8U)
            if image.max() == 1:
                image *= 255
            if image.dtype != np.uint8:
                image = image.astype(np.uint8)
            # Convert to CV_8UC1 format
            if len(image.shape) > 2:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            converted_images.append(image)
        
        contours = []
        for img in converted_images:

            contours_list,_ = cv2.findContours(img, cv2.CHAIN_APPROX_SIMPLE, cv2.CHAIN_APPROX_NONE)
            contours.append(contours_list)

        return contours
    
    def apply_thresholds(self, contours: np.array, 
                         min_area : int = 20, 
                         max_area : int = 400,
                         min_round_ratio : float = 0.5,
                         max_round_ratio: float = 2.0):
        
        filtered_contours = []       
        for contours_list in contours:
            for contours_list in contours:

                new_contours_list = []
                for cntr in contours_list:
                    
                    area = cv2.contourArea(cntr)
                    # contour_areas.append(area)

                    arclength = cv2.arcLength(cntr, True)
                    # arclengths.append(arclength)
                    
                    round_ratio =  4 * np.pi * area / ( arclength**2 )
                    # roundness.append(round_ratio)

                    if (round_ratio > 0.5) and (area > 20) and (area < 400):
                        new_contours_list.append(cntr)
                
            
                filtered_contours.append(new_contours_list)
        
        return filtered_contours

In [7]:
imgs_dir = '../data/train/image/*'
imgs_paths = glob(imgs_dir)
imgs_paths = sorted(imgs_paths)


# Create list with all images in gray scale
images_gray = [ cv2.imread(image_path, cv2.IMREAD_GRAYSCALE) for image_path in imgs_paths]
images_gray = images_gray[:42]
# Visualizando imagens
# plot_images(images_gray)

In [8]:
detector = CoringDetector(images_gray)
transformed_images  = detector.apply_transforms()


AttributeError: 'CoringDetector' object has no attribute 'images'